# Лабораторная работа №2
Группа 3530903/70301  
Шергалис Донат, Ижко Георгий, Сидорова Елена

# Last FM hometask <br>
https://www.kaggle.com/ravichaubey1506/lastfm <br>
1. Выбрать данные по странам своей группы (совместно): <br>
    3530203_70101: Germany, Netherlands <br>
    3530203_70102: Belarus, Ukraine, Poland, Russian Federation<br>
    **3530903_70301: Sweden, Finland, Norway, Denmark, Iceland**<br>
    3530903_70302: Spain, Portugal, France, Italy, Belgium<br>
    
2. Попытаться найти полезные с точки зрения продвижения групп (или еще чего-нибудь) и нетривиальные правила, используя алгоритмы Apriori, FPGrowth, FPMax и всевозможные метрики. Хотя бы 5 правил.
3. Вывести эти правила в отдельных ячейках. 
4. Подумать, как можно было бы использовать полученные правила.

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, fpgrowth, association_rules, fpmax

In [3]:
data = pd.read_csv("lastfm.csv")
data.head()

,user,artist,sex,country
0,1,red hot chili peppers,f,Germany
1,1,the black dahlia murder,f,Germany
2,1,goldfrapp,f,Germany
3,1,dropkick murphys,f,Germany
4,1,le tigre,f,Germany


In [4]:
countries = 'Sweden, Finland, Norway, Denmark, Iceland'.split(', ')
countries

['Sweden', 'Finland', 'Norway', 'Denmark', 'Iceland']

In [5]:
lastfm_df = data[data['country'].isin(countries)]
len(lastfm_df)

30268

In [6]:
user_info = lastfm_df[['user', 'country', 'sex']].drop_duplicates('user')
user_info.set_index('user', inplace=True)
user_info.head()

,country,sex
user,,
5,Finland,m
7,Finland,m
19,Sweden,f
28,Sweden,m
48,Norway,m


In [7]:
# dummy encoding исполнителя
pivoted_artist = lastfm_df[['user', 'artist']].pivot_table(index='user', columns='artist',aggfunc=any, fill_value=False).astype(int)
print("Количество слушателей по исполнителю")
pivoted_artist.sum()

Количество слушателей по исполнителю


artist
...and you will know us by the trail of dead    17
2pac                                            49
3 doors down                                    58
30 seconds to mars                              57
311                                              3
                                                ..
yeah yeah yeahs                                 14
yellowcard                                      15
yo la tengo                                      8
zero 7                                          23
Édith piaf                                      14
Length: 995, dtype: int64

In [8]:
# dummy encoding страны
pivoted_country = lastfm_df[['user', 'country']].pivot_table(index='user', columns='country',aggfunc=any, fill_value=False).astype(int)
pivoted_country.sum()

country
Denmark    119
Finland    510
Iceland     16
Norway     214
Sweden     575
dtype: int64

In [9]:
# dummy encoding пола пользователя
pivoted_sex = lastfm_df[['user', 'sex']].pivot_table(index='user', columns='sex',aggfunc=any, fill_value=False).astype(int)
pivoted_sex.sum()

sex
f     332
m    1102
dtype: int64

In [10]:
pivoted_all = pd.concat([pivoted_artist, pivoted_country, pivoted_sex], axis=1)
pivoted_all.head()

,...and you will know us by the trail of dead,2pac,3 doors down,30 seconds to mars,311,36 crazyfists,44,50 cent,65daysofstatic,[unknown],...,yo la tengo,zero 7,Édith piaf,Denmark,Finland,Iceland,Norway,Sweden,f,m
user,,,,,,,,,,,,,,,,,,,,,
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
19,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,1,0
28,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


# Apriori

## Apriori for artists

In [11]:
apriori_artist = apriori(pivoted_artist, min_support=0.03, use_colnames=True)
apriori_artist

,support,itemsets
0,0.034170,(2pac)
1,0.040446,(3 doors down)
2,0.039749,(30 seconds to mars)
3,0.038354,(50 cent)
4,0.042538,(a perfect circle)
...,...,...
257,0.036960,"(the beatles, queen)"
258,0.032775,"(the beatles, radiohead)"
259,0.036262,"(rammstein, system of a down)"
260,0.033473,"(red hot chili peppers, system of a down)"


### Apriori by confidence for artists

In [12]:
apriori_artist_rules = association_rules(apriori_artist, metric="confidence", min_threshold=0.4)
apriori_artist_rules.sort_values('confidence', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
10,(soilwork),(in flames),0.060669,0.154812,0.039052,0.643678,4.157813,0.029659,2.371980
15,(sonata arctica),(nightwish),0.076011,0.133194,0.040446,0.532110,3.995005,0.030322,1.852586
2,(children of bodom),(in flames),0.078103,0.154812,0.039052,0.500000,3.229730,0.026960,1.690377
13,(koЯn),(system of a down),0.072524,0.137378,0.036262,0.500000,3.639594,0.026299,1.725244
8,(koЯn),(in flames),0.072524,0.154812,0.034170,0.471154,3.043399,0.022943,1.598174
12,(iron maiden),(metallica),0.092050,0.164575,0.043236,0.469697,2.854006,0.028087,1.575374


### Apriori by support for artists

In [13]:
apriori_artist_rules = association_rules(apriori_artist, metric="support", min_threshold=0.03)
apriori_artist_rules.sort_values('support', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
33,(in flames),(metallica),0.154812,0.164575,0.057183,0.369369,2.244388,0.031705,1.324746
32,(metallica),(in flames),0.164575,0.154812,0.057183,0.347458,2.244388,0.031705,1.295224
42,(in flames),(system of a down),0.154812,0.137378,0.049512,0.319820,2.328029,0.028244,1.268226
43,(system of a down),(in flames),0.137378,0.154812,0.049512,0.360406,2.328029,0.028244,1.321445
15,(muse),(coldplay),0.134589,0.172245,0.049512,0.367876,2.135764,0.026330,1.309481
14,(coldplay),(muse),0.172245,0.134589,0.049512,0.287449,2.135764,0.026330,1.214526


### Apriori by lift for artists

In [14]:
apriori_artist_rules = association_rules(apriori_artist, metric="lift", min_threshold=0.03)
apriori_artist_rules.sort_values('lift', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
38,(in flames),(soilwork),0.154812,0.060669,0.039052,0.252252,4.157813,0.029659,1.256213
39,(soilwork),(in flames),0.060669,0.154812,0.039052,0.643678,4.157813,0.029659,2.371980
54,(pink floyd),(led zeppelin),0.110181,0.073222,0.033473,0.303797,4.149005,0.025405,1.331191
55,(led zeppelin),(pink floyd),0.073222,0.110181,0.033473,0.457143,4.149005,0.025405,1.639140
75,(sonata arctica),(nightwish),0.076011,0.133194,0.040446,0.532110,3.995005,0.030322,1.852586
74,(nightwish),(sonata arctica),0.133194,0.076011,0.040446,0.303665,3.995005,0.030322,1.326931


### Apriori by leverage for artists

In [15]:
apriori_artist_rules = association_rules(apriori_artist, metric="leverage", min_threshold=0.03)
apriori_artist_rules.sort_values('leverage', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(metallica),(in flames),0.164575,0.154812,0.057183,0.347458,2.244388,0.031705,1.295224
1,(in flames),(metallica),0.154812,0.164575,0.057183,0.369369,2.244388,0.031705,1.324746
2,(nightwish),(sonata arctica),0.133194,0.076011,0.040446,0.303665,3.995005,0.030322,1.326931
3,(sonata arctica),(nightwish),0.076011,0.133194,0.040446,0.532110,3.995005,0.030322,1.852586


### Apriori by conviction for artists

In [16]:
apriori_artist_rules = association_rules(apriori_artist, metric="conviction", min_threshold=0.03)
apriori_artist_rules.sort_values('conviction', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
39,(soilwork),(in flames),0.060669,0.154812,0.039052,0.643678,4.157813,0.029659,2.371980
75,(sonata arctica),(nightwish),0.076011,0.133194,0.040446,0.532110,3.995005,0.030322,1.852586
52,(koЯn),(system of a down),0.072524,0.137378,0.036262,0.500000,3.639594,0.026299,1.725244
4,(children of bodom),(in flames),0.078103,0.154812,0.039052,0.500000,3.229730,0.026960,1.690377
55,(led zeppelin),(pink floyd),0.073222,0.110181,0.033473,0.457143,4.149005,0.025405,1.639140
8,(children of bodom),(nightwish),0.078103,0.133194,0.036262,0.464286,3.485789,0.025859,1.618038


## Apriori for artists, sex and country

In [17]:
apriori_artist_rules = association_rules(apriori_artist, metric="support", min_threshold=0.03)
apriori_artist_rules.sort_values('support', ascending=False).head(6)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
33,(in flames),(metallica),0.154812,0.164575,0.057183,0.369369,2.244388,0.031705,1.324746
32,(metallica),(in flames),0.164575,0.154812,0.057183,0.347458,2.244388,0.031705,1.295224
42,(in flames),(system of a down),0.154812,0.137378,0.049512,0.319820,2.328029,0.028244,1.268226
43,(system of a down),(in flames),0.137378,0.154812,0.049512,0.360406,2.328029,0.028244,1.321445
15,(muse),(coldplay),0.134589,0.172245,0.049512,0.367876,2.135764,0.026330,1.309481
14,(coldplay),(muse),0.172245,0.134589,0.049512,0.287449,2.135764,0.026330,1.214526


In [18]:
apriori_all = apriori(pivoted_all, min_support=0.03, use_colnames=True)
apriori_all

,support,itemsets
0,0.034170,(2pac)
1,0.040446,(3 doors down)
2,0.039749,(30 seconds to mars)
3,0.038354,(50 cent)
4,0.042538,(a perfect circle)
...,...,...
545,0.054393,"(system of a down, m, Finland)"
546,0.042538,"(system of a down, m, Sweden)"
547,0.048815,"(the beatles, m, Sweden)"
548,0.038354,"(the killers, m, Sweden)"


### Apriori by confidence for artists, sex and country

In [19]:
apriori_all_rules = association_rules(apriori_all, metric="confidence", min_threshold=0.1)
print("found", len(apriori_all_rules), "rules")
apriori_all_rules.sort_values('confidence', ascending=False).head(5)

found 635 rules


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
88,(dire straits),(m),0.040446,0.76848,0.039749,0.982759,1.278835,0.008667,13.428173
310,(scooter),(m),0.044630,0.76848,0.042538,0.953125,1.240273,0.008241,4.939098
503,"(iron maiden, Finland)",(m),0.042538,0.76848,0.040446,0.950820,1.237274,0.007756,4.707578
286,(rage against the machine),(m),0.069038,0.76848,0.065551,0.949495,1.235550,0.012497,4.584100
497,"(nightwish, iron maiden)",(m),0.034868,0.76848,0.032775,0.940000,1.223194,0.005980,3.858670


Правила не очень интересные, попробуем найти правила с несколькими consequents:

In [20]:
apriori_all_rules[apriori_all_rules['consequents'].map(lambda x: len(x) > 1)] \
    .sort_values('confidence', ascending=False) \
    .head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
396,(amorphis),"(m, Finland)",0.046722,0.273361,0.032775,0.701493,2.566174,0.020003,2.434240
520,(lamb of god),"(m, Finland)",0.052301,0.273361,0.032078,0.613333,2.243673,0.017781,1.879238
476,(soilwork),"(in flames, m)",0.060669,0.132497,0.034868,0.574713,4.337568,0.026829,2.039805
425,(children of bodom),"(m, Finland)",0.078103,0.273361,0.041841,0.535714,1.959730,0.020491,1.565068
607,(soilwork),"(m, Finland)",0.060669,0.273361,0.030683,0.505747,1.850106,0.014099,1.470176


Используя алгоритм apriori для исполнителей, можно сделать следующие выводы: <br>
* по метрике confidence: те, кто слушают soilwork, children of bodom, koЯn также слушают и in flames, интересно, что все группы играют в стиле метал, а soilwork, children of bodom, in flames осонованы в скандинасвких странах
* по метрике support: слушающие in flames также слушают и metallica (обратное тоже справедливо), аналогичные выводы для system of a down и in flames, а также для muse и coldplay
* по метрике lift: аналогично выводам по метрике support наблюдается связь между прослушиванием групп in flames и soilwork, led zeppelin и pink floyd, sonata arctica и nightwish
* по метрике leverage: подтверждаются выводы по метрике  lift
* по метрике conviction: soilwork->in flames (как и по confidence), sonata arctica->nightwish (lift), koЯn->system of a down, children of bodom->in flames(обратная связь по сравнению по confidence), led zeppelin->pink floyd(lift)	


Используя алгоритм apriori для всех колонок, можно сделать следующие выводы: <br>
* по метрике confidence: товарищи слушающие dire straits, scooter, rage against the machine, (nightwish & iron maiden) в основном - мужчины; 
также, люди из страны Finland слушающие iron maiden в основном тоже мужчины
* люди слушающие amorphis, lamb of god, children of bodom, soilwork в основном - мужчины из страны Finland; люди слушающие soilwork являются мужчинами и при этом еще слушают in flames

# FP-growth

fp-growth позволяет установить меньший порог и получить больше наборов, среди которых могут быть интересные правила

In [21]:
fpgrowth_artist = fpgrowth(pivoted_artist, min_support=0.005, use_colnames=True)
len(fpgrowth_artist)

14331

### FP-growth by confidence for artists

In [22]:
fpgrowth_artist_rules = association_rules(fpgrowth_artist, metric="confidence", min_threshold=0.8)
fpgrowth_artist_rules.sort_values('confidence', ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
977,"(2pac, dmx, kanye west)","(snoop dogg, the game)",0.005579,0.013250,0.005579,1.0,75.473684,0.005505,inf
300,"(50 cent, jay-z, nas)",(kanye west),0.006974,0.081590,0.006974,1.0,12.256410,0.006405,inf
318,"(eminem, jay-z, nas, snoop dogg)",(kanye west),0.006276,0.081590,0.006276,1.0,12.256410,0.005764,inf
644,"(metallica, nightwish, amon amarth)",(children of bodom),0.005579,0.078103,0.005579,1.0,12.803571,0.005143,inf
314,"(eminem, jay-z, nas)",(kanye west),0.007671,0.081590,0.007671,1.0,12.256410,0.007045,inf
310,"(eminem, 50 cent, nas)",(snoop dogg),0.005579,0.034868,0.005579,1.0,28.680000,0.005384,inf
649,"(dark tranquillity, in flames, hypocrisy)",(soilwork),0.006974,0.060669,0.006974,1.0,16.482759,0.006550,inf
308,"(2pac, 50 cent, nas)",(snoop dogg),0.005579,0.034868,0.005579,1.0,28.680000,0.005384,inf
650,"(dark tranquillity, hypocrisy, soilwork)",(in flames),0.006974,0.154812,0.006974,1.0,6.459459,0.005894,inf
303,"(snoop dogg, 50 cent, jay-z, nas)",(kanye west),0.006276,0.081590,0.006276,1.0,12.256410,0.005764,inf


Используя алгоритм FP-growth для исполнителей, можно сделать следующие  общие выводы по метрике confidence: <br>
* прослушивающие 2pac, dmx, eminem, 50 cent, nas также прослушивают и snoop dogg <br>
* прослушивающие eminem, jay-z, nas, snoop dogg, 50 cent также прослушивают и kanye west

# FPmax

FP-Max вариант FP-Growth, который ориентируется на получение максимального набора элементов.

In [31]:
fpmax_artist = fpmax(pivoted_artist, min_support=0.0012, use_colnames=True)
len(fpmax_artist)

64929

In [32]:
fpmax_artist

,support,itemsets
0,0.001395,"(the white stripes, radiohead, pearl jam, chic..."
1,0.001395,"(zero 7, beastie boys, boards of canada, moby,..."
2,0.001395,(lacrimosa)
3,0.001395,"(beatsteaks, blur, the beatles)"
4,0.001395,(late of the pier)
...,...,...
64924,0.001395,"(kent, system of a down, red hot chili peppers..."
64925,0.001395,"(kent, metallica, radiohead, muse, coldplay)"
64926,0.001395,"(metallica, radiohead, system of a down, in fl..."
64927,0.001395,"(muse, system of a down, in flames, the beatles)"


### FPmax by support for artists

Не получилось использовать алгоритм FPmax из-за слишклм низкого support

In [ ]:
#fpmax_artist_rules = association_rules(fpmax_artist, support_only=True, min_threshold=0.001)
#fpmax_artist_rules.sort_values('support', ascending=False).head(10)

Зависимости, найденные в данной работе будет очень полезно применить в рекомендательных системах, а также для составления списка проигрываемых мелодий на радиостанциях